# summarize the results form different experiments for my paper manuscript

In [1]:
import numpy as np 
import pandas as pd
import mlflow
from mlflow import MlflowClient

## compare the X3D_m from scratch with from pre-trained weights of videos

In [35]:
# common hyper parameters: X3D_m, four-output abs PFT values, 
# from scratch: 2667, from pre-trained weights: 2665
id = 2643
merge_type = 'ensemble'
mode = 'test'  #  or 'test'

# retrive the run for the ex id
mlflow.set_tracking_uri("http://nodelogin02:5000")
experiment = mlflow.set_experiment("lung_fun_db15")
client = MlflowClient()
run = client.search_runs(experiment_ids=[experiment.experiment_id], filter_string=f"params.id = '{str(id)}'")[0]


In [36]:
keys = ['DLCOcPP', 'FEV1PP', 'FVCPP', 'TLCPP']
summarize = get_metrics(run, keys = keys, mode='valid').astype(float).round(2)
summarize


,DLCOcPP,FEV1PP,FVCPP,TLCPP
icc,0.30,0.47,0.50,0.53
r,0.32,0.47,0.51,0.54
mae,17.07,14.88,14.98,13.59
me,3.33,2.59,3.15,3.40
me_std,21.30,19.11,19.27,17.09
mre,0.29,0.18,0.18,0.17


In [4]:
runss = client.search_runs(experiment_ids=[experiment.experiment_id], filter_string=f"tags.mlflow.parentRunId = '{run.info.run_id}'")  # 4 days after after the parent ID created 


In [5]:
len(runss)

4

In [6]:
def child_run_id(parent_id: str):
    client = MlflowClient()
    parent_run = client.search_runs(experiment_ids=[experiment.experiment_id], filter_string=f"params.id = '{str(parent_id)}'")[0]
    runss = client.search_runs(experiment_ids=[experiment.experiment_id], filter_string=f"tags.mlflow.parentRunId = '{parent_run.info.run_id}'")  # 4 days after after the parent ID created 
    
    if len(runss) == 4:  # all folds are there
        out = {int(ru.data.params['fold']): int(ru.data.params['id']) for ru in runss}
        out[0 ] = parent_id
        print(f"successfully found four folds runs for {parent_id}: {out}")
        return out
    else:
        tmp = {int(ru.data.params['fold']): int(ru.data.params['id']) for ru in runss}
        print(f"the searched runs for {parent_id} do not 4 folds, only has {len(runss)} folds: {tmp}")
        return None
    
     

In [40]:
child_run_wt_id_dt = child_run_id(id)

successfully found four folds runs for 2534: {4: 2588, 3: 2563, 2: 2553, 1: 2535, 0: 2534}


In [41]:
child_run_wt_id_dt

{4: 2588, 3: 2563, 2: 2553, 1: 2535, 0: 2534}

In [9]:
keys = ['DLCOc_SB', 'FEV1', 'FVC', 'TLC_He']

In [27]:
def get_metrics(run, keys, mode):
    metric_ls = ['icc', 'r', 'mae', 'me', 'me_std', 'mre']
    df = pd.DataFrame(columns = keys, index = metric_ls)
    for key in keys:
        try:
            for metric in metric_ls:
                col_name = f"{merge_type}_{metric}_{mode}_{key}"  # like: ensemble_icc_test
                
                df[key][metric] = float(run.data.params[col_name])
        except:
            pass
    return df

    

In [21]:
summarize = get_metrics(run, keys = keys).astype(float).round(2)
summarize

,DLCOc_SB,FEV1,FVC,TLC_He
icc,0.48,0.52,0.53,0.58
r,0.56,0.57,0.58,0.66
mae,1.34,0.55,0.68,0.83
